# Style Transfer

<img src="https://i0.wp.com/chelseatroy.com/wp-content/uploads/2018/12/neural_style_transfer.png?resize=768%2C311&ssl=1">

La idea de este trabajo final es reproducir el siguiente paper:

https://arxiv.org/pdf/1508.06576.pdf

El objetivo es transferir el estilo de una imagen dada a otra imagen distinta. 

Como hemos visto en clase, las primeras capas de una red convolucional se activan ante la presencia de ciertos patrones vinculados a detalles muy pequeños.

A medida que avanzamos en las distintas capas de una red neuronal convolucional, los filtros se van activando a medida que detectan patrones de formas cada vez mas complejos.

Lo que propone este paper es asignarle a la activación de las primeras capas de una red neuronal convolucional (por ejemplo VGG19) la definición del estilo y a la activación de las últimas capas de la red neuronal convolucional, la definición del contenido.

La idea de este paper es, a partir de dos imágenes (una que aporte el estilo y otra que aporte el contenido) analizar cómo es la activación de las primeras capas para la imagen que aporta el estilo y cómo es la activación de las últimas capas de la red convolucional para la imagen que aporta el contenido. A partir de esto se intentará sintetizar una imagen que active los filtros de las primeras capas que se activaron con la imagen que aporta el estilo y los filtros de las últimas capas que se activaron con la imagen que aporta el contenido.

A este procedimiento se lo denomina neural style transfer.

# En este trabajo se deberá leer el paper mencionado y en base a ello, entender la implementación que se muestra a continuación y contestar preguntas sobre la misma.

# Una metodología posible es hacer una lectura rápida del paper (aunque esto signifique no entender algunos detalles del mismo) y luego ir analizando el código y respondiendo las preguntas. A medida que se planteen las preguntas, volviendo a leer secciones específicas del paper terminará de entender los detalles que pudieran haber quedado pendientes.

Lo primero que haremos es cargar dos imágenes, una que aporte el estilo y otra que aporte el contenido. A tal fin utilizaremos imágenes disponibles en la web.

In [62]:
# Imagen para estilo
!wget https://upload.wikimedia.org/wikipedia/commons/5/52/La_noche_estrellada1.jpg

# Imagen para contenido
!wget https://upload.wikimedia.org/wikipedia/commons/thumb/f/f4/Neckarfront_T%C3%BCbingen_Mai_2017.jpg/775px-Neckarfront_T%C3%BCbingen_Mai_2017.jpg

# Creamos el directorio para los archivos de salida
!mkdir /content/output

zsh:1: command not found: wget
zsh:1: command not found: wget
mkdir: /content: No such file or directory


In [28]:
from keras.preprocessing.image import load_img, save_img, img_to_array
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
import time
import argparse

from keras.applications import vgg19
from keras import backend as K
from pathlib import Path

In [29]:
# Definimos las imagenes que vamos a utilizar, y el directorio de salida

base_image_path = Path("./prueba_propia/contenido.jpg")
style_reference_image_path = Path("./prueba_propia/estilo.jpg")
result_prefix = Path("./prueba_propia/output")
iterations = 100

# 1) En base a lo visto en el paper ¿Qué significan los parámetros definidos en la siguiente celda?

Respuesta:

En el algoritmo del paper se propone que la loss este compuesta por dos partes. Una de ellas siendo la loss correspondiente al contenido de la una de las imagenes y la otra al estilo de la otra imagen. Tambien dentro de esta loss general se agregan factores mediante los cuales se puede pesar cuanta importantancia (peso) se le da a cada una de dichas losses (contenido y estilo).
Estos valores "style_weight" y "content_weight", refieren al peso que tendrá cada parte de la loss, en este caso, la loss del estilo tendrá mayor peso que la del contenido.

Por otro lado, "total_variation_weight" pesa la componente de la loss que se usa para medir el suavizado o cambios muy bruscos en la imagen

In [30]:
total_variation_weight = 0.1
style_weight = 50
content_weight = 1

In [31]:
# Definimos el tamaño de las imágenes a utilizar
width, height = load_img(base_image_path).size
img_nrows = 400
img_ncols = int(width * img_nrows / height)

# 2) Explicar qué hace la siguiente celda. En especial las últimas dos líneas de la función antes del return. ¿Por qué?

Ayuda: https://keras.io/applications/

Respuesta:

*- concepto general:*

El proposito general es preprocesar la imagen para que este en el formato correcto para que pueda procesarse por al red VGG19
(por ejemplo cargar la imagen, convertirla en un numpy array, pasarla a BGR y centrar cada canal de color en 0).

*- load_img:*

Carga la imagen que del "image_path", con el size dato por "img_nrows" e "img_ncols" y devuelve una instancia de la imagen en formato Python Imaging Library

*- img_to_array:*

Toma una imagen en formato PIL (python imaging library) y la convierte en un numpy array de 3 dimensiones (alto, ancho y RGB)

*- np.expand_dims:*

Agrega un nuevo axis en la posicion indicada, en este caso en axis = 0 (osea, fila)

*- vgg19.preprocess_input:*

Preprocesa el numpy array de la imagen obtenida y la convierte de RGB a BGR, luego cada uno de los canales respectivos a cada color es centrado en 0 para que sean acordes a los datasets y forma de entremaniento de VGG19 con ImageNet.


In [32]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_nrows, img_ncols))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

# 3) Habiendo comprendido lo que hace la celda anterior, explique de manera muy concisa qué hace la siguiente celda. ¿Qué relación tiene con la celda anterior?

Respuesta:

Esta funcion quita el axis que se agrego para el procesamiento, dejando un formato normal de imagen,
tambien descentra los canales de media 0 a una media dada por cada canal de color y luego modifica la imagend e BGR a RGB.

Hace esto para volver del formato que necesita VGG19 al formato standard de imagenes

In [33]:
def deprocess_image(x):
    x = x.reshape((img_nrows, img_ncols, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [34]:
# get tensor representations of our images
# K.variable convierte un numpy array en un tensor, para 
base_image = K.variable(preprocess_image(base_image_path))
style_reference_image = K.variable(preprocess_image(style_reference_image_path))

In [35]:
combination_image = K.placeholder((1, img_nrows, img_ncols, 3))

Aclaración:

La siguiente celda sirve para procesar las tres imagenes (contenido, estilo y salida) en un solo batch.

In [36]:
# combine the 3 images into a single Keras tensor
input_tensor = K.concatenate([base_image,
                              style_reference_image,
                              combination_image], axis=0)

In [37]:
# build the VGG19 network with our 3 images as input
# the model will be loaded with pre-trained ImageNet weights
model = vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet', include_top=False)
print('Model loaded.')

# get the symbolic outputs of each "key" layer (we gave them unique names).
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

Model loaded.


In [38]:
model.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

# 4) En la siguientes celdas:

- ¿Qué es la matriz de Gram?¿Para qué se usa?

La matriz gram es la herramienta que se utiliza para medir la correlacion entre features, de esta forma puede diferenciarse el estilo del contenido.
Es el producto interno entre los vectores de feature de una capa dada.

Para calcular la matriz gram se debe tomar la representacion de una capa dada L y convertila en una matriz de dos dimensiones.
Cada fila de dicha matriz esta compuesta de toda la salida de los filtros de la capa anterior. Siendo que cada filtro busca un feature en específico, podria decirse que cada fila de la matriz representa un feature dado en distintas posiciones de la imagen

Para generar una textura que coincida con la de una imagen dada, se debe usar descenso por gradiente sobre la imagen de salida (imagen compuesta por ruido blaco unicamente), para encontrar otra imagen que corresponda con la representacion del estilo de la imagen origina (osea, los features de la imagen del estilo).
Esto se hace minimizando el MSE entre las matrices de Gram de la imagen original y la imagen a ser generada.

Para generar la imagen que combine el contenido de una imagen con el estilo de la otra, se intentan minizar en conjunto los MSE de las matrices de ambos componentes (MSE sobre matrices de gram para estilo y MSE regular para contenido), pesando cada uno de estos (contenido y estilo) para darles la preponderancia que se prefiera.

- ¿Por qué se permutan las dimensiones de x?

Se permutan las dimensiones para que que en la matriz de gram quede el producto punto por cada uno de los canales RGB. (y finalmente la matriz de gram quede de 3 x 3)

In [39]:
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

# 5) Losses:

Explicar qué mide cada una de las losses en las siguientes tres celdas.

Rta:

*-style_loss:*

Mide la loss del estilo, esto se calcula como la sumatoria de el MSE entre la matriz de gram de una imagen del estilo y la matriz de gram de la imagen generada, divido un valor en funcion de los cuadrados de la cantidad de canales y tamaño de la imagen.

*-content_loss:*

Mide el MSE del contenido a modificar (imagen output con rudio blanco) y la imagen dada como input sobre la cual se debe tomar el contenido.

*-total_variation_loss:*

Esta componente de la loss busca evitar cambios bruscos (componentes de alta frecuencia) en la imagen, logrando un suavizado en la imagen.

In [40]:
def style_loss(style, combination):
    assert K.ndim(style) == 3
    assert K.ndim(combination) == 3
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return K.sum(K.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

In [41]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))


In [42]:
def total_variation_loss(x):
    assert K.ndim(x) == 4
    a = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :])
    b = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))


In [43]:
# Armamos la loss total
loss = K.variable(0.0)
layer_features = outputs_dict['block5_conv2']
base_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss = loss + content_weight * content_loss(base_image_features,
                                            combination_features)

feature_layers = ['block1_conv1', 'block2_conv1',
                  'block3_conv1', 'block4_conv1',
                  'block5_conv1']
for layer_name in feature_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :] 
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss = loss + (style_weight / len(feature_layers)) * sl
loss = loss + total_variation_weight * total_variation_loss(combination_image)

In [44]:
grads = K.gradients(loss, combination_image)

outputs = [loss]
if isinstance(grads, (list, tuple)):
    outputs += grads
else:
    outputs.append(grads)

f_outputs = K.function([combination_image], outputs)

# 6) Explique el propósito de las siguientes tres celdas. ¿Qué hace la función fmin_l_bfgs_b? ¿En qué se diferencia con la implementación del paper? ¿Se puede utilizar alguna alternativa?

Respuesta:

*- Explicacion de las celdas:*

eval_loss_and_grads es una funcion que obtiene tanto la loss como el gradiente que se definieron en las celdas anteriores.
esta funcion es utilizada dentro del evaluator para obtener ambos valores (loss y gradient) y guardarlos en el mismo.
Aunque estrictamente solo se necesite la loss en la funcion de loss, siendo que fmin_l_bfgs_b necesita dos funciones diferentes para loss y grad, se calculan ambos (loss y grad) en la funcion de loss y luego se guardan en el evaluator, para posteriormente poder devolver el gradiente en la funcion que corresponda.
Calcular estos valores en dos funciones separadas seria ineficiente.

la ultima celda instancia el evaluator, carga y preprocesa la imagen base que tiene el contenido y luego por cada una de las iteraciones esperadas (en nuestor caso 100) utiliza la funcion fmin_l_bfgs_b para poder ir minimizando la loss de la y aplicando el estilo.

*- Funcion fmin_l_bfgs_b:*

Es una funcion que busca minimizar una funcion pasada por parametro usando el algoritmo L-BFGS-B.
en el uso que le damos, se le pasa la funcion de loss a minimizar, la imagen preprosesada y hecho un flatten para ser un array y la fprima que indica cual es el gradiente a obtener para minimizar la funcion.

*- Diferencia con el paper:*

- Se agrega total_variation_loss.
- El ratio del pesaje de style vs content es diferente.
- Solo se toma 1 capa convolucional para medir el MSE del content y no se intenta ver con cada una de las capaz, en nuestro caso solo tomamos la segunda capa convolucional del quinto bloque, en el paper hacen la prueba con la capa 1 de los 5 bloques. 
- VGG19 usa max pooling, en el paper se utilizó average pooling
- En el paper se utiliza descenso por gradiente para la imagen a generar, aqui se usa fmin_l_bfgs_b

*- Alternativas:*

Como alternativas podria cambiarse la loss que se utilzia para el contenido, tambien podria usarse alguna otra arquitectura que no sea VGG19

In [45]:
def eval_loss_and_grads(x):
    x = x.reshape((1, img_nrows, img_ncols, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values

# this Evaluator class makes it possible
# to compute loss and gradients in one pass
# while retrieving them via two separate functions,
# "loss" and "grads". This is done because scipy.optimize
# requires separate functions for loss and gradients,
# but computing them separately would be inefficient.

In [46]:
class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

# 7) Ejecute la siguiente celda y observe las imágenes de salida en cada iteración.

In [ ]:
evaluator = Evaluator()

# run scipy-based optimization (L-BFGS) over the pixels of the generated image
# so as to minimize the neural style loss
x = preprocess_image(base_image_path)

for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    # save current generated image
    img = deprocess_image(x.copy())
    fname = result_prefix / ('output_at_iteration_%d.png' % i)
    save_img(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

Start of iteration 0
Current loss value: 434102170000.0
Image saved as prueba_propia/output/output_at_iteration_0.png
Iteration 0 completed in 387s
Start of iteration 1
Current loss value: 154287010000.0
Image saved as prueba_propia/output/output_at_iteration_1.png
Iteration 1 completed in 402s
Start of iteration 2
Current loss value: 87032980000.0
Image saved as prueba_propia/output/output_at_iteration_2.png
Iteration 2 completed in 369s
Start of iteration 3
Current loss value: 60093575000.0
Image saved as prueba_propia/output/output_at_iteration_3.png
Iteration 3 completed in 327s
Start of iteration 4
Current loss value: 45415404000.0
Image saved as prueba_propia/output/output_at_iteration_4.png
Iteration 4 completed in 318s
Start of iteration 5
Current loss value: 37144445000.0
Image saved as prueba_propia/output/output_at_iteration_5.png
Iteration 5 completed in 1336s
Start of iteration 6
Current loss value: 31288824000.0
Image saved as prueba_propia/output/output_at_iteration_6.pn

# 8) Generar imágenes para distintas combinaciones de pesos de las losses. Explicar las diferencias. (Adjuntar las imágenes generadas como archivos separados.)

Respuesta:

*- Test1:*

total_variation_weight = 0.1

style_weight = 50

content_weight = 1

imagen:
<img src="./content_50_1_0_1/output/output_at_iteration_99.png">

conclusion:

En esta prueba se le da mayor peso al estilo que al contenido y es notable como desde las primeras iteraciones, ya se logra una presencia mas importante del estilo en las imagenes generadas comparandolo con las imagenes que propone la notebook.

*- Test2:*

total_variation_weight = 100

style_weight = 10

content_weight = 1

imagen:
<img src="./content_10_1_100/output/output_at_iteration_99.png">

conclusion:

Es notorio como en esta iteracion se suavizan los los cambios entre pixeles y los colores quedan mucho mas vibrantes, la imagen empieza a parecer blureada y con trazos mucho mas anchos que los de la imagen del estilo

*- Test3:*

total_variation_weight = 0.1

style_weight = 5

content_weight = 100

imagen:
<img src="./content_5_100_0_1/output/output_at_iteration_99.png">

conclusion:

Al subir el peso del contenido, se nota como el estilo se hace mucho menos presente.

*- Test3:*

total_variation_weight = 1

style_weight = 100

content_weight = 4

imagen:
<img src="./content_4_100_1/output/output_at_iteration_99.png">

conclusion:

Es notable como al subir todos los valores multiplicadores de loss, no suben todos de la misma forma, sino lo que se ve es que lo importante es el ratio que estos generan entre si.
en este test, se logró un estilo similar al de la prueba, con trazos mas anchos por el totla variation weight mayor.


# 9) Cambiar las imágenes de contenido y estilo por unas elegidas por usted. Adjuntar el resultado.

Respuesta:

imagen de contenido:

<img src="./prueba_propia/contenido.jpg">

imagen de estilo:

<img src="./prueba_propia/estilo.jpg">

creacion final:

<img src="./prueba_propia/output/output_at_iteration_99.png">

total_variation_weight = 0.1

style_weight = 50

content_weight = 1

conclusion:

Se nota claramente como se toma el estilo y el color de la imagen que contiene el estilo, logrando un gran cambio y similitud a esta, manteniendo el contenido de la imagen correspondiente